In [1]:
import csv
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

data_list = []

with open("csv/Pumpkin_Seeds_Dataset.csv", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    
    next(reader)
    for row in reader:
        data_list.append(row)

data = np.array(data_list)

features = data[:, :-1].astype(float)
labels = data[:, -1]
labels = [0 if label[-1] == 'k' else 1 for label in labels]

# normalisation
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

all_ids = np.arange(0, data.shape[0])
random_seed = 1
# training : test = 2000 : 500, real_training : validation = 1600 : 400 
rem_set_ids, test_set_ids = train_test_split(all_ids, test_size=0.2, train_size=0.8,
                                 random_state=random_seed, shuffle=True)
train_set_ids, val_set_ids = train_test_split(rem_set_ids, test_size=0.2, train_size=0.8,
                                 random_state=random_seed, shuffle=True)

training_features = features[train_set_ids, :]
training_labels = [labels[i] for i in train_set_ids]

test_features = features[test_set_ids, :]
test_labels = [labels[i] for i in test_set_ids]

val_features = features[val_set_ids, :]
val_labels = [labels[i] for i in val_set_ids]

In [179]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

random.seed(random_seed)
torch.manual_seed(random_seed)
numpy.random.seed(random_seed)

class FNN(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, output_size):
        super().__init__()
        
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_sizes)):
            if i == 0:
                self.hidden_layers.append(nn.Linear(input_size, hidden_sizes[i]))
            else:
                self.hidden_layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
        
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        for hidden_layer in self.hidden_layers:
            x = torch.relu(hidden_layer(x))
        output = self.output_layer(x)
        return output

# transfer the numpy data type to torch tensor type
class MetDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx, :], self.labels[idx]

In [180]:
features_count = training_features.shape[1]

# Setting hyperparameters
p_layers = [10, 20, 30]
num_epochs = 20
learning_rates = [0.001, 0.01, 0.1]
# input_matrix(batch_size, features_count), output_matrix(hidden_size[-1], output_size)
batch_sizes = [16, 32, 64]

parameters = []
for batch_size in batch_sizes:
    for rate in learning_rates:
        for p_layer in p_layers:
            x = (p_layer, rate, batch_size)
            parameters.append(x)

In [181]:
def calculate_accuracy(parameter):
    
    p_layer, rate, batch_size = parameter
    
    hidden_size = [p_layer, features_count]
    output_size = np.unique(training_labels).shape[0]
    
    three_layer_MLP = FNN(features_count, hidden_size, output_size)
    
    # loading data by batch
    train_set = MetDataset(training_features, training_labels)
    train_dataloader = DataLoader(train_set, batch_size=batch_size)
    
    val_set = MetDataset(val_features, val_labels)
    val_dataloader = DataLoader(val_set, batch_size=len(val_set))
    
    # Setting up the stochastic gradient descent optimizer for updating the model weights
    optimizer = optim.SGD(three_layer_MLP.parameters(), lr=rate)
    
    # optimizer = optim.SGD(three_layer_MLP.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        
        three_layer_MLP.train()
        
        for batch, (x_train, y_train) in enumerate(train_dataloader):
            # The gradient of the parameter is zeroed. In PyTorch, gradients are cumulative.
            optimizer.zero_grad()
            
            train_pred = three_layer_MLP.forward(x_train)
            train_loss = loss_function(train_pred, y_train)
            
            train_loss.backward()
            optimizer.step()
        
            # Evaluating on the validation set
            val_accuracies = []
            three_layer_MLP.eval()
            for batch, (x_val, y_val) in enumerate(val_dataloader):
                val_pred = three_layer_MLP.forward(x_val)
                val_loss = loss_function(val_pred, y_val)
                val_f1_scores, val_accuracy = my_metrics(y_val, val_pred)
                val_accuracies.append(val_accuracy)

            avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)

    return avg_val_accuracy

In [182]:
accuracies = []
for parameter in parameters:
    accuracies.append((parameter, calculate_accuracy(parameter)))
accuracies_sorted = sorted(accuracies, key=lambda x : x[1], reverse=True)
print(accuracies_sorted)
print('\n', accuracies_sorted[0])

[((20, 0.1, 16), 0.8625), ((30, 0.1, 64), 0.85), ((20, 0.01, 16), 0.8475), ((10, 0.1, 16), 0.8475), ((20, 0.1, 64), 0.8475), ((30, 0.01, 32), 0.845), ((30, 0.1, 32), 0.845), ((30, 0.01, 16), 0.8425), ((10, 0.1, 32), 0.8425), ((10, 0.01, 16), 0.84), ((20, 0.1, 32), 0.84), ((10, 0.1, 64), 0.8375), ((30, 0.1, 16), 0.8275), ((20, 0.01, 32), 0.81), ((10, 0.01, 32), 0.7025), ((10, 0.001, 16), 0.59), ((30, 0.001, 16), 0.5825), ((30, 0.001, 32), 0.5725), ((20, 0.001, 64), 0.5675), ((30, 0.001, 64), 0.5625), ((10, 0.01, 64), 0.5625), ((10, 0.001, 32), 0.555), ((20, 0.001, 16), 0.5525), ((20, 0.01, 64), 0.5525), ((30, 0.01, 64), 0.5525), ((10, 0.001, 64), 0.51), ((20, 0.001, 32), 0.4475)]

 ((20, 0.1, 16), 0.8625)


In [184]:
best_parameter = accuracies_sorted[0][0]

calculate_accuracy(best_parameter)
test_set = MetDataset(test_features, test_labels)
test_dataloader = DataLoader(test_set, batch_size=len(test_set))
test_accuracies = []
three_layer_MLP.eval()
for batch, (x_test, y_test) in enumerate(test_dataloader):
    test_pred = three_layer_MLP.forward(x_test)
    test_loss = loss_function(test_pred, y_test)
    test_f1_scores, test_accuracy = my_metrics(y_test, test_pred)
    test_accuracies.append(test_accuracy)
avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)
print(avg_test_accuracy)

0.83
